# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os 

from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Capstone-Project'

experiment=Experiment(ws, experiment_name)

### Creating Cluster 

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "Computecluster"
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    print("Found exisitng compute cluster")
except ComputeTargetException:
    print("Creating new computer cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    print("Created computer cluster")

Creating new computer cluster
Created computer cluster


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--n_est': choice(10, 20, 30, 40, 50),
    '--min_samples_split': choice(1, 2, 3)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', entry_script='train.py', compute_target=compute_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                    hyperparameter_sampling=param_sampling,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=12,
                                    max_concurrent_runs=4,
                                    policy=early_termination_policy)

In [7]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2ea787b0-25e1-4021-95a2-a6a3c7ef1db2
Web View: https://ml.azure.com/experiments/Capstone-Project/runs/HD_2ea787b0-25e1-4021-95a2-a6a3c7ef1db2?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-138594/workspaces/quick-starts-ws-138594

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-11T19:00:55.824259][API][INFO]Experiment created<END>\n""<START>[2021-02-11T19:00:56.438143][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-11T19:00:56.749965][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-11T19:00:56.8918232Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2021-02-11T19:01:27.4541249Z][SCHEDULER][INFO]The execution environment was successfully prepared.<END><START>[2021-02-11T19:01:27.3910851Z][SCHEDULER][INFO]Scheduling job, id='HD_2ea787b

{'runId': 'HD_2ea787b0-25e1-4021-95a2-a6a3c7ef1db2',
 'target': 'Computecluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-11T19:00:55.589031Z',
 'endTimeUtc': '2021-02-11T19:14:04.232522Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name":

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f"Best Run id: {best_run.id}")
print(f"Accuracy: {best_run_metrics['Accuracy']}")
print(f"Number of estimators: {parameter_values[1]}")
print(f"Number of Iterations: {parameter_values[3]}")

Best Run id: HD_2ea787b0-25e1-4021-95a2-a6a3c7ef1db2_5
Accuracy: 0.8611111111111112
Number of estimators: 3
Number of Iterations: 10


In [12]:
#TODO: Save the best model
best_run.register_model(model_name='HyperdriveModel', model_path='.')

Model(workspace=Workspace.create(name='quick-starts-ws-138594', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-138594'), name=HyperdriveModel, id=HyperdriveModel:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service